A partir del archivo csv Case, determine las tres ciudades con más casos confirmados de la enfermedad. La salida debe contener tres columnas: provincia, ciudad y casos confirmados. El resultado debe contener exactamente los tres nombre de ciudades con más casos confirmados ya que no se admiten otros valores.



Cree un dataframe a partir del archivo csv PatientInfo. Asegúrese de que su dataframe no contenga pacientes duplicados.



¿Cuántos pacientes tienen informado por quién se contagiaron(columna infected_by)? Obtenga solo los pacientes que tengan informado por quién se contagiaron.



A partir de la salida del inciso anterior obtenga solo los pacientes femeninos. La salida no debe contener las columnas released_date y deceased_date.



Establezca el número de particiones del dataframe resultante del inciso anterior en dos. Escriba el dataframe resultante en un archivo parquet. La salida debe estar particionada por la provincia y el modo de escritura debe ser overwrite.

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import NullType

spark = SparkSession.builder.getOrCreate()

In [84]:
df_Cases = spark.read.options(header='true', inferSchema='true').csv('../datasets/Case.csv')
df_Cases.show(5, truncate=False)

+--------+--------+------------+-----+---------------------------+---------+---------+----------+
| case_id|province|city        |group|infection_case             |confirmed|latitude |longitude |
+--------+--------+------------+-----+---------------------------+---------+---------+----------+
|1000001 |Seoul   |Yongsan-gu  |true |Itaewon Clubs              |139      |37.538621|126.992652|
|1000002 |Seoul   |Gwanak-gu   |true |Richway                    |119      |37.48208 |126.901384|
|1000003 |Seoul   |Guro-gu     |true |Guro-gu Call Center        |95       |37.508163|126.884387|
|1000004 |Seoul   |Yangcheon-gu|true |Yangcheon Table Tennis Club|43       |37.546061|126.874209|
|1000005 |Seoul   |Dobong-gu   |true |Day Care Center            |43       |37.679422|127.044374|
+--------+--------+------------+-----+---------------------------+---------+---------+----------+
only showing top 5 rows



In [85]:
df_Cases.dtypes

[(' case_id', 'int'),
 ('province', 'string'),
 ('city', 'string'),
 ('group', 'boolean'),
 ('infection_case', 'string'),
 ('confirmed', 'int'),
 ('latitude', 'string'),
 ('longitude', 'string')]

In [89]:
dfClean = df_Cases.filter((F.col("city") != "-") & (F.col("city") != "from other city"))
dfClean.show(1)

+--------+--------+----------+-----+--------------+---------+---------+----------+
| case_id|province|      city|group|infection_case|confirmed| latitude| longitude|
+--------+--------+----------+-----+--------------+---------+---------+----------+
| 1000001|   Seoul|Yongsan-gu| true| Itaewon Clubs|      139|37.538621|126.992652|
+--------+--------+----------+-----+--------------+---------+---------+----------+
only showing top 1 row



In [131]:
## TOP 3 CON MAS CONTAGIO SALIDA 3 COLUMNAS PROVINCIA,CIUDAD,CASOS CONFIMADO
top_3 = dfClean.select("province","city","confirmed").orderBy(F.col("confirmed").desc()).show()

+-----------------+------------+---------+
|         province|        city|confirmed|
+-----------------+------------+---------+
|            Daegu|      Nam-gu|     4511|
|            Daegu|Dalseong-gun|      196|
|            Seoul|  Yongsan-gu|      139|
|            Daegu|      Seo-gu|      124|
|            Seoul|   Gwanak-gu|      119|
| Gyeongsangbuk-do|Cheongdo-gun|      119|
|Chungcheongnam-do|  Cheonan-si|      103|
|            Daegu|Dalseong-gun|      101|
|            Seoul|     Guro-gu|       95|
| Gyeongsangbuk-do| Bonghwa-gun|       68|
|      Gyeonggi-do| Seongnam-si|       67|
|      Gyeonggi-do|  Bucheon-si|       67|
| Gyeongsangbuk-do|Gyeongsan-si|       66|
|      Gyeonggi-do|Uijeongbu-si|       50|
|            Seoul|Yangcheon-gu|       43|
|            Seoul|   Dobong-gu|       43|
|            Seoul|     Guro-gu|       41|
| Gyeongsangbuk-do|  Yechun-gun|       40|
|            Busan|  Dongnae-gu|       39|
|            Daegu|     Dong-gu|       39|
+----------

In [35]:
patientDF = spark.read.options(header='true', inferSchema='true').csv('../datasets/Patientinfo.csv')

In [36]:
patientDF.show(5, truncate=False)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|sex   |age|country|province|city       |infection_case      |infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|state   |
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|male  |50s|Korea  |Seoul   |Gangseo-gu |overseas inflow     |NULL       |75            |2020-01-22        |2020-01-23    |2020-02-05   |NULL         |released|
|1000000002|male  |30s|Korea  |Seoul   |Jungnang-gu|overseas inflow     |NULL       |31            |NULL              |2020-01-30    |2020-03-02   |NULL         |released|
|1000000003|male  |50s|Korea  |Seoul   |Jongno-gu  |contact with patient|2002000001 |17            |NULL              |2020-01-30    |2020-0

In [37]:
patientDF.dtypes

[('patient_id', 'bigint'),
 ('sex', 'string'),
 ('age', 'string'),
 ('country', 'string'),
 ('province', 'string'),
 ('city', 'string'),
 ('infection_case', 'string'),
 ('infected_by', 'string'),
 ('contact_number', 'string'),
 ('symptom_onset_date', 'string'),
 ('confirmed_date', 'date'),
 ('released_date', 'date'),
 ('deceased_date', 'date'),
 ('state', 'string')]

In [45]:
patientcleanDF = patientDF.distinct()

In [53]:
patientcleanDF = patientcleanDF.filter(F.col("infected_by") != "NULL")
patientcleanDF.show(5, truncate=False)

+----------+------+----+-------+-----------------+--------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|sex   |age |country|province         |city          |infection_case      |infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|state   |
+----------+------+----+-------+-----------------+--------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|2000000198|female|50s |Korea  |Gyeonggi-do      |Bucheon-si    |contact with patient|2000000167 |NULL          |NULL              |2020-03-13    |NULL         |NULL         |isolated|
|2000000775|male  |30s |Korea  |Gyeonggi-do      |Gwangmyeong-si|contact with patient|2000000763 |NULL          |NULL              |2020-05-27    |NULL         |NULL         |isolated|
|2000000797|female|10s |Korea  |Gyeonggi-do      |Gimpo-si      |contact wi

In [133]:
patientcleanDF.count()

1346

In [101]:
# A partir de la salida del inciso anterior obtenga solo los pacientes femeninos. La salida no debe contener las columnas released_date y deceased_date.
patientFemenine = patientcleanDF.filter((F.col("sex") == "female") & (F.col("sex").isNotNull())).drop('released_date','deceased_date')

In [ ]:
# Establezca el número de particiones del dataframe resultante del inciso anterior en dos. Escriba el dataframe resultante en un archivo parquet. La salida debe estar particionada por la provincia y el modo de escritura debe ser overwrite.

patientFemenine.coalesce(2).write.partitionBy('province').mode("overwrite").parquet("../datasets/partition/")